# Milestone 1
In the notebook below I have created a pipeline for generating artificial data to be used in training for Telescope image finding on finder images. Thus far I have better learned how to use os, how to write to JSON files, and image processing using pillow.

The pipeline works as follows:
- using RA and Dec coordinates to retrieve 60'x 60' finder images SkyView 
- put any generated images in the *finder_images* folder
- create the telescope image by croping the finder image, fliping the image horizontally, increasing contrast, adding gaussian noise
- add each of these images to the *telescope_images* folder
- write the path of the finder image, telescope image, and area of the crop to a JSON file 

# Generate finder charts 

In [2]:
import os
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy import units as u
from astroquery.skyview import SkyView
import matplotlib.pyplot as plt

#the following block of code was written with the help of chatGPT

# --------------------------------------------------
# 1. Output folder
# --------------------------------------------------
dirname = "finder_images"
os.makedirs(dirname, exist_ok=True)

# --------------------------------------------------
# 2. Coordinates (example list)
# --------------------------------------------------
coords = [
    SkyCoord("05h35m17.3s", "-05d23m28s"),  # M42
    SkyCoord("00h42m44.3s", "+41d16m09s"),  # M31
    SkyCoord("12h51m26.3s", "+27d07m42s"),  # M94
]

# --------------------------------------------------
# 3. Function to generate an DSS finder chart
# --------------------------------------------------
def make_dss_finder(coord, name, fov_arcmin=60):
    radius = (fov_arcmin / 2) * u.arcmin
    images = SkyView.get_images(
        position=coord,
        survey=["DSS2 Red"],
        radius=radius,
        pixels=1200  # Try for large field, SkyView may limit resolution
    )

    hdu = images[0][0]
    wcs = WCS(hdu.header)

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={"projection": wcs})
    ax.imshow(hdu.data, cmap='gray', origin='lower')

    ax.axis('off')

    return fig

# --------------------------------------------------
# 4. Loop through targets
# --------------------------------------------------
for i, coord in enumerate(coords, start=1):
    name = f"target_{i:04d}"
    png_path = os.path.join(dirname, f"{name}.png")

    fig = make_dss_finder(coord, name, fov_arcmin=60)
    fig.savefig(png_path, dpi=150, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    if i == 1 or i % 10 == 0:
        print(f"Generated DSS finder: {png_path}")

#end of where chatGPT helped 


Generated DSS finder: finder_images\target_0001.png


# Genrate telescope images 

In [34]:
from PIL import Image, ImageEnhance
import random 

import numpy as np
 

def make_telescope_img(input_path):
    try:
        img = Image.open(input_path) 
    except FileNotFoundError:
        print("Error: Image file not found. Please check the path.")
        exit()

        
    # --------------------------------------------------
    #crop image
    # --------------------------------------------------
    width, height = img.size
    t_width = round(0.25*width)
    t_height = round(0.25*height)
    
    x_coord = random.randint(0, width - t_width)
    y_coord = random.randint(0, height - t_height)

    crop_area = (x_coord, y_coord, x_coord + t_width, y_coord + t_height)
    cropped_img = img.crop(crop_area)

    
    # --------------------------------------------------
    #flip image 
    # --------------------------------------------------
    flipped_img = cropped_img.transpose(Image.FLIP_LEFT_RIGHT)

    
    # --------------------------------------------------
    #increases contrast of image
    # --------------------------------------------------
    enhancer = ImageEnhance.Contrast(flipped_img)
    enhanced_image = enhancer.enhance(random.uniform(1,2))

    
    # --------------------------------------------------
    #add noise to image 
    # --------------------------------------------------
    noise_image = np.array(enhanced_image.convert("RGB"), dtype=np.float32)
    # Generate 2 dimensional Gaussian noise 
    noise = np.random.normal(loc=0.0, scale=10, size=noise_image.shape[:2])
    #stack noise to return to rgb 
    noise = np.stack((noise,noise,noise), axis = 2)
    # Add noise + clip to [0,255]
    noisy = np.clip(noise_image + noise, 0, 255).astype(np.uint8)

    return Image.fromarray(noisy), crop_area


# Write data to JSON file 

In [37]:
import json 
import os

# --------------------------------------------------
# 1. Output folder
# --------------------------------------------------
dirname = "telescope_images"
os.makedirs(dirname, exist_ok=True)

output_json = os.path.join(dirname, "data.json")
all_data = []   # list to store metadata for each image


counter_dict = {}

# --------------------------------------------------
# loop through finder images 
# --------------------------------------------------
for filename in os.listdir("finder_images"):

    if filename.endswith(".png"):
        base_name = filename[:-4]
        full_path = os.path.join("finder_images", filename)
        print(f"Processing PNG file: {full_path}")

        if base_name not in counter_dict:
            counter_dict[base_name] = 1

        # Generate telescope image
        telescope_img, crop_area = make_telescope_img(full_path)

        # Create file name: (findername)_telescope_####
        count = counter_dict[base_name]
        out_name = f"{base_name}_telescope_{count:04d}.png"
        out_path = os.path.join(dirname, out_name)

        # Save the telescope image
        telescope_img.save(out_path)
        print(f"Saved: {out_path}")

        # Increment counter for that finder image
        counter_dict[base_name] += 1

        #JSON data for telescope image 
        data = {
            "finder_path": full_path,
            "telescope_path": out_path,
            "crop_area": crop_area
        }
        all_data.append(data)


# --------------------------------------------------
# put data in JSON file 
# --------------------------------------------------
with open(output_json, "w") as f:
    json.dump(all_data, f, indent=4)


Processing PNG file: finder_images\target_0001.png
Saved: telescope_images\target_0001_telescope_0001.png
Processing PNG file: finder_images\target_0002.png
Saved: telescope_images\target_0002_telescope_0001.png
Processing PNG file: finder_images\target_0003.png
Saved: telescope_images\target_0003_telescope_0001.png
